In [ ]:
!pip install huggingface
!pip -q install -q nltk transformers
!pip install sec-api
!pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pandas_datareader as web
from google.colab import drive
import requests
import os
import nltk
from transformers import pipeline
nltk.download('punkt')
from transformers import BertTokenizer, BertForSequenceClassification
import time
import threading
import re
from datetime import datetime
from sec_api import QueryApi
from datetime import datetime, timedelta
from pynvml import *
from sec_api import QueryApi
from sec_api import ExtractorApi
import numpy as np
import matplotlib.pyplot as plt
from dateutil.parser import parse
import statsmodels.api as sm
import statsmodels.api as sm
import pandas as pd
from google.colab import drive
import scipy.stats as stats
import numpy as np
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import seaborn as sns
import nltk

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [1]:
#@title S&P-500 Ticker List

total_SP = ['A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF.B',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMY',
 'BR',
 'BRK.B',
 'BRO',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDAY',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COST',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTLT',
 'CTRA',
 'CTSH',
 'CTVA',
 'CVS',
 'CVX',
 'CZR',
 'D',
 'DAL',
 'DD',
 'DE',
 'DFS',
 'DG',
 'DGX',
 'DHI',
 'DHR',
 'DIS',
 'DISH',
 'DLR',
 'DLTR',
 'DOV',
 'DOW',
 'DPZ',
 'DRI',
 'DTE',
 'DUK',
 'DVA',
 'DVN',
 'DXC',
 'DXCM',
 'EA',
 'EBAY',
 'ECL',
 'ED',
 'EFX',
 'EIX',
 'EL',
 'ELV',
 'EMN',
 'EMR',
 'ENPH',
 'EOG',
 'EPAM',
 'EQIX',
 'EQR',
 'EQT',
 'ES',
 'ESS',
 'ETN',
 'ETR',
 'ETSY',
 'EVRG',
 'EW',
 'EXC',
 'EXPD',
 'EXPE',
 'EXR',
 'F',
 'FANG',
 'FAST',
 'FCX',
 'FDS',
 'FDX',
 'FE',
 'FFIV',
 'FIS',
 'FISV',
 'FITB',
 'FLT',
 'FMC',
 'FOX',
 'FOXA',
 'FRT',
 'FSLR',
 'FTNT',
 'FTV',
 'GD',
 'GE',
 'GEN',
 'GILD',
 'GIS',
 'GL',
 'GLW',
 'GM',
 'GNRC',
 'GOOG',
 'GOOGL',
 'GPC',
 'GPN',
 'GRMN',
 'GS',
 'GWW',
 'HAL',
 'HAS',
 'HBAN',
 'HCA',
 'HD',
 'HES',
 'HIG',
 'HII',
 'HLT',
 'HOLX',
 'HON',
 'HPE',
 'HPQ',
 'HRL',
 'HSIC',
 'HST',
 'HSY',
 'HUM',
 'HWM',
 'IBM',
 'ICE',
 'IDXX',
 'IEX',
 'IFF',
 'ILMN',
 'INCY',
 'INTC',
 'INTU',
 'INVH',
 'IP',
 'IPG',
 'IQV',
 'IR',
 'IRM',
 'ISRG',
 'IT',
 'ITW',
 'IVZ',
 'J',
 'JBHT',
 'JCI',
 'JKHY',
 'JNJ',
 'JNPR',
 'JPM',
 'K',
 'KDP',
 'KEY',
 'KEYS',
 'KHC',
 'KIM',
 'KLAC',
 'KMB',
 'KMI',
 'KMX',
 'KO',
 'KR',
 'L',
 'LDOS',
 'LEN',
 'LH',
 'LHX',
 'LIN',
 'LKQ',
 'LLY',
 'LMT',
 'LNC',
 'LNT',
 'LOW',
 'LRCX',
 'LUMN',
 'LUV',
 'LVS',
 'LW',
 'LYB',
 'LYV',
 'MA',
 'MAA',
 'MAR',
 'MAS',
 'MCD',
 'MCHP',
 'MCK',
 'MCO',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MGM',
 'MHK',
 'MKC',
 'MKTX',
 'MLM',
 'MMC',
 'MMM',
 'MNST',
 'MO',
 'MOH',
 'MOS',
 'MPC',
 'MPWR',
 'MRK',
 'MRNA',
 'MRO',
 'MS',
 'MSCI',
 'MSFT',
 'MSI',
 'MTB',
 'MTCH',
 'MTD',
 'MU',
 'NCLH',
 'NDAQ',
 'NDSN',
 'NEE',
 'NEM',
 'NFLX',
 'NI',
 'NKE',
 'NOC',
 'NOW',
 'NRG',
 'NSC',
 'NTAP',
 'NTRS',
 'NUE',
 'NVDA',
 'NVR',
 'NWL',
 'NWS',
 'NWSA',
 'NXPI',
 'O',
 'ODFL',
 'OGN',
 'OKE',
 'OMC',
 'ON',
 'ORCL',
 'ORLY',
 'OTIS',
 'OXY',
 'PARA',
 'PAYC',
 'PAYX',
 'PCAR',
 'PCG',
 'PEAK',
 'PEG',
 'PEP',
 'PFE',
 'PFG',
 'PG',
 'PGR',
 'PH',
 'PHM',
 'PKG',
 'PKI',
 'PLD',
 'PM',
 'PNC',
 'PNR',
 'PNW',
 'POOL',
 'PPG',
 'PPL',
 'PRU',
 'PSA',
 'PSX',
 'PTC',
 'PWR',
 'PXD',
 'PYPL',
 'QCOM',
 'QRVO',
 'RCL',
 'RE',
 'REG',
 'REGN',
 'RF',
 'RHI',
 'RJF',
 'RL',
 'RMD',
 'ROK',
 'ROL',
 'ROP',
 'ROST',
 'RSG',
 'RTX',
 'SBAC',
 'SBUX',
 'SCHW',
 'SEDG',
 'SEE',
 'SHW',
 'SIVB',
 'SJM',
 'SLB',
 'SNA',
 'SNPS',
 'SO',
 'SPG',
 'SPGI',
 'SRE',
 'STE',
 'STLD',
 'STT',
 'STX',
 'STZ',
 'SWK',
 'SWKS',
 'SYF',
 'SYK',
 'SYY',
 'T',
 'TAP',
 'TDG',
 'TDY',
 'TECH',
 'TEL',
 'TER',
 'TFC',
 'TFX',
 'TGT',
 'TJX',
 'TMO',
 'TMUS',
 'TPR',
 'TRGP',
 'TRMB',
 'TROW',
 'TRV',
 'TSCO',
 'TSLA',
 'TSN',
 'TT',
 'TTWO',
 'TXN',
 'TXT',
 'TYL',
 'UAL',
 'UDR',
 'UHS',
 'ULTA',
 'UNH',
 'UNP',
 'UPS',
 'URI',
 'USB',
 'V',
 'VFC',
 'VICI',
 'VLO',
 'VMC',
 'VRSK',
 'VRSN',
 'VRTX',
 'VTR',
 'VTRS',
 'VZ',
 'WAB',
 'WAT',
 'WBA',
 'WBD',
 'WDC',
 'WEC',
 'WELL',
 'WFC',
 'WHR',
 'WM',
 'WMB',
 'WMT',
 'WRB',
 'WRK',
 'WST',
 'WTW',
 'WY',
 'WYNN',
 'XEL',
 'XOM',
 'XRAY',
 'XYL',
 'YUM',
 'ZBH',
 'ZBRA',
 'ZION',
 'ZTS',]

#Data Cleaning & Engineering


In [ ]:
# This function inputs the total text of the filing, extracts the period for which it was filed, and
# returns the period ending year and month (yyyy-mm)
def get_period(url):

  response = requests.get(url,headers={"User-Agent": "Mozilla/5.0"})
  text = response.text
  text = text.replace('\n', '')
  text = text.replace('\t','')
  date = re.findall(r"CONFORMED PERIOD OF REPORT:\s*(\d{8})", text)

  if date:
      date = date[0]
      year = date[0:4]
      month = date[4:6]
      day = date[6:8]
      formatted_date = year + '-' + month + '-' + day

  else:
    formatted_date = 'FLAG'

  return formatted_date

In [ ]:
# Creates a file for each company in the S&P containing all 10-Q/Ks MD&A sections


for ticker in total_SP:
  df = pd.DataFrame(columns=['Ticker', 'Form Type','Period Ended','MD&A','url'])

  query = {
      "query": {
          "query_string": {
              "query": "(formType:\"10-Q\" OR formType:\"10-K\") AND ticker:("+ ticker +")"
          }
      },
      "from": "0",
      "size": "1000",
      "sort": [{ "filedAt": { "order": "desc" } }]
  }

  response = queryApi.get_filings(query)
  filings = response['filings']

  for index, doc in enumerate(filings,start=0):

    print(ticker,index)

    # Extract
    doc_type = doc['formType']
    url = doc['linkToTxt']

    # Load
    df.loc[index, 'Ticker'] = ticker
    df.loc[index, 'Form Type'] = doc_type
    df.loc[index,'url'] = url

    # Extract Period Date
    period = doc.get('periodOfReport')
    if period:
      df.loc[index, 'Period Ended'] = period
    else:
      df.loc[index, 'Period Ended'] = get_period(url)
    
    if doc_type == '10-K':
      # Extract 10k mdna
      text = extractApi.get_section(url,"7","text")
      df.loc[index,'MD&A'] = text

    elif doc_type == '10-Q':
      # Extract 10-Q mdna
      text = extractApi.get_section(url,"part1item2","text")
      df.loc[index,'MD&A'] = text
    
  df.to_csv('/content/drive/MyDrive/Capstone-DataSets/Filings/'+ticker+'.csv')
  print(ticker + " file saved")

In [ ]:
def getPercentReturn(old_val,new_val):

    percent_change = ((new_val - old_val)/old_val) * 100
    return percent_change

class DateError(Exception):
    pass

def get_nearest_row(priceData,date):
  temp = priceData.loc[priceData['Date'] == date]
  if temp.empty:
    count = 0
    while temp.empty:
        date = pd.to_datetime(date)
        date = date + timedelta(days=1)
        date = date.strftime('%Y-%m-%d')
        temp = priceData.loc[priceData['Date'] == date]
        count += 1
        if count > 15:
          raise DateError('Matching Date not in dataset')
  return temp

def getPeriodReturn(numDays,priceData, curRowIndex):
  if len(priceData) - 1 >= curRowIndex + numDays:
    old_price = priceData.loc[curRowIndex,'Adj Close']
    new_price = priceData.loc[curRowIndex + numDays,'Adj Close']
    return getPercentReturn(old_price, new_price)
  else:
    return "NO DATA" 

In [ ]:
getPercentReturn(1,2)

100.0

In [ ]:
# Inputs DF containing filings and their respective sentiment scores
# For each document, this method calculates the return over 90, 60, 30, 10, & 5 subsequent days and adds values into new columns
dateAnom = []
temp = pd.DataFrame()
def append_returns(ticker):

    priceData = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/PriceData/'+ticker+'.csv',index_col=0)
    filing = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Two_S_NewDate/'+ticker+'-2C.csv',index_col=0)

    # Remove timestamp from date, if present
    priceData['Date'] = priceData['Date'].str.split(' ', n=1, expand=True)[0]

    # Create new columns for returns. Note the conversion between *calendar* days and *trading* days. 
    # One calendar month == 21 Trading days
    
    # 252 Trading Days
    filing['365 Day Return'] = None

    # 126 Trading Days
    filing['180 Day Return'] = None

    # 63 Trading Days
    filing['90 Day Return'] = None

    # 42 Trading days
    filing['60 Day Return'] = None

    # 21 Trading Days
    filing['30 Day Return'] = None

    # 10 Trading Days
    filing['10 Day Return'] = None

    # 5 Trading Days
    filing['5 Day Return'] = None


    for index, doc_row in filing.iterrows():
      print(index)
      # Return calculations must start from date filed, NOT quarter end 
      # start_date = doc_row['Period Ended']
      start_date = doc_row['Filed At']
      print("FILED AT: ", start_date)
      

      # Get row from price data closest to filing date (sometimes irregularities with filings)
      # If no price data, continue to next row. Very few occurences of this, hence the skip.
      try:
        matching_start_row = get_nearest_row(start_date,priceData)
      except DateError as d:
        continue
        
      

      # Get price from date closest to filing date
      start_price = matching_start_row['Adj Close']
      print("START DATE FOR PRICE DATA: ", matching_start_row['Date'])

      # Get index of the corresponding row
      i = matching_start_row.index[0]
      print("STARTING PRICE: ", matching_start_row['Adj Close'][i])


      filing.loc[index,'365 Day Return'] = getPeriodReturn(252,priceData,i)
      filing.loc[index,'180 Day Return'] = getPeriodReturn(126,priceData,i)
      filing.loc[index,'90 Day Return'] = getPeriodReturn(63,priceData,i)
      filing.loc[index,'60 Day Return'] = getPeriodReturn(42,priceData,i)
      filing.loc[index,'30 Day Return'] = getPeriodReturn(21,priceData,i)
      filing.loc[index,'10 Day Return'] = getPeriodReturn(10,priceData,i)
      filing.loc[index,'5 Day Return'] = getPeriodReturn(5,priceData,i)
    
    filing = filing.reindex(columns=['Ticker', 'Form Type', 'Period Ended','Filed At','S-yiyang','S-FLS', '365 Day Return','180 Day Return', '90 Day Return','60 Day Return','30 Day Return','10 Day Return', '5 Day Return','MD&A','url'])
    
    filing.to_csv('/content/drive/MyDrive/Capstone-DataSets/Two_S_NewDate/'+ticker+'-2C.csv')
    print(ticker,' FILE SAVED')

    
    
    

In [ ]:
# Put all sentiment scores into single list
scores = []
for t in total_SP:
  temp = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Two_Sentiment/'+t+'-2C.csv',index_col=0)
  for index, row in temp.iterrows():
    scores.append(row['S-FLS'])

In [ ]:
# Histogram of all sentiment values
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(1, 1, 1)

ax.hist(scores, bins=70, color='orange', edgecolor='black')
ax.set_xlabel('Value')
ax.set_ylabel('Frequency')
ax.set_title('Total FLS Sentiment')

ax.grid(True)
ax.set_axisbelow(True)

plt.show()

In [ ]:
# Histogram of all negative sentiment values
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(1, 1, 1)

ax.hist(neg, bins=70, color='orange', edgecolor='black')
ax.set_xlabel('Value')
ax.set_ylabel('Frequency')
ax.set_title('Negative FLS Sentiment Values')

ax.grid(True)
ax.set_axisbelow(True)

plt.show()

In [ ]:
# Histogram of all positive sentiment values
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(1, 1, 1)

ax.hist(pos, bins=70, color='orange', edgecolor='black')
ax.set_xlabel('Value')
ax.set_ylabel('Frequency')
ax.set_title('Positive Sentiment Values')

ax.grid(True)
ax.set_axisbelow(True)

plt.show()

In [ ]:
# Anomaly: Some SEC files go back further than the ticker data. Tested multiple apis and websites which all match the ticker data. 
# Not sure where these filigns are coming from. They consist of 744/37694 (~2%) of all documents

revisit = pd.DataFrame()
count = 0
tickers = set()
for t in total_SP:
  temp = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Completed/'+t+'-C.csv')
  for index, row in temp.iterrows():
    if pd.isnull(row['90 Day Return']) & pd.isnull(row['60 Day Return']) & pd.isnull(row['30 Day Return']) & pd.isnull(row['10 Day Return']) & pd.isnull(row['5 Day Return']):
      count += 1
      tickers.add(t)



In [ ]:
# Create and populate dictionary to sort tickers by GICS industry:

no_price_data = ['FRC', 'GEHC', 'SBNY']

industry_dict = {'Industrials':[],
             'Health Care':[],
             'Information Technology':[],
             'Communication Services':[],Communication
             'Consumer Staples':[],
             'Consumer Discretionary':[],
             'Financials':[],
             'Materials':[],
             'Utilities':[],
             'Real Estate': [],
             'Energy':[]
             }

industry_csv = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/S&P-industries.csv',index_col=0)

for index, row in industry_csv.iterrows():
  sector = str(row['Sector'])
  ticker = str(row['Tickers'])
  if ticker in no_price_data:
    continue 

  industry_dict[sector].append(ticker)

# Veryify correct number of tickers - should be 500
keys = list(industry_dict.keys())
count = 0
for k in keys:
  count += len(industry_dict[k])

count

500

In [ ]:
# Create histogram for sentiment of each industry
keys = list(industry_dict.keys())
for k in keys:
  scores = []
  for t in industry_dict[k]:
    temp = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Two_Sentiment/'+t+'-2C.csv',index_col=0)
    for index, row in temp.iterrows():
      scores.append(row['S-FLS'])


  fig = plt.figure(figsize=(12, 10))
  ax = fig.add_subplot(1, 1, 1)

  ax.hist(scores, bins=70, color='orange', edgecolor='black')
  ax.set_xlabel('Value')
  ax.set_ylabel('Frequency')
  ax.set_title(k+' FLS Sentiment')

  ax.grid(True)
  ax.set_axisbelow(True)

  plt.show()

count


In [ ]:
# Input: URL, Output: new column value with filing date

total_SP = 'A'
for ticker in total_SP:
  

  query = {
      "query": {
          "query_string": {
              "query": "(formType:\"10-Q\" OR formType:\"10-K\") AND ticker:("+ ticker +")"
          }
      },
      "from": "0",
      "size": "1000",
      "sort": [{ "filedAt": { "order": "asc" } }]
  }

  response = queryApi.get_filings(query)

  test = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Two_Sentiment/'+ticker+'-2C.csv',index_col=0)
  
  test['Filed At'] = None


  for index, row in test.iterrows():

    # Get df index of first url to match
    matching_url = test['url'][index]

    for i in range(len(response['filings'])):
      if response['filings'][i]['linkToTxt'] == matching_url:
        
        filed_at = response['filings'][i]['filedAt']
        filed_at_formatted = parse(filed_at).date()
        test.loc[index, 'Filed At'] = filed_at_formatted
        break
    
  

  test.to_csv('/content/drive/MyDrive/Capstone-DataSets/Two_S_NewDate/'+ticker+'-2C.csv')



  
  

In [ ]:
# Method inputs score type (either S-yiyang or S-FLS) winsorizes scores to be maximally within 3STDs and derives z scores
# Z score derivation is done using mean and std calculations performed exluding 0s

def calc_z_scores(score_type):

  master = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/master.csv',index_col = 0)

  # Calculate sentiment score mean and std for winsorization
  scores = np.array(master[score_type])
  mean = np.mean(scores)
  std = np.std(scores)
  
  # Winsorize Scores to be maximally 3 stds from mean
  lower = mean - (3 * std)
  upper = mean + (3 * std)
  
  # winsorized_data = winsorize(scores, limits=(lower,upper))

  upperCount = 0
  lowerCount = 0

  # Winsorize outliers 
  for val in scores:
    if val > upper:
      val = upper
      upperCount +=1
    elif val < lower:
      val = lower
      lowerCount+=1
  
  print("UPPER COUNT: ", upperCount)
  print("LOWER COUNT: ", lowerCount)

  # Remove 0's to calculate mean and std for z scores
  no_zeros = scores[scores != 0]
  no_zero_mean = np.mean(no_zeros)
  no_zero_std = np.std(no_zeros)

  # Loop over winsorized data array to calculate a z score for each value
  z_scores = []
  for x in scores:
    z = (x - no_zero_mean) / no_zero_std
    z_scores.append(z)
  
  assert(len(z_scores) == len(master))

  if score_type == 'S-FLS':
    master['Z-Score-FLS'] = z_scores
  
  elif score_type == 'S-yiyang':
    master['Z-Score'] = z_scores

  master.to_csv('/content/drive/MyDrive/Capstone-DataSets/master-v1.1.csv')  

In [ ]:
# Method inputs a period, and score type (either S-yiyang or S-FLS) and performs a linear regression 
def calc_t_score(period,score_type):
  
  master = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/master-v1.1.csv',index_col=0)
  col = str(period) + ' Day Return'

  df = pd.DataFrame()
  df['Returns'] = master[col]
  df['Z-Scores'] = master[score_type]

  df['Returns'] = pd.to_numeric(df.Returns, errors='coerce')
  df = df.dropna()

  x = np.array(df['Z-Scores'])
  x = sm.add_constant(x)
  
  y = np.array(df['Returns'])


  model = sm.OLS(y,x).fit()
  print(model.summary())
  # tvals = list(model.tvalues)

  # if len(tvals) == 1:
  #   return tvals[0]
  # else:
  #   return tvals[1]
  return model

In [ ]:

periods = [365,180,90,60,30,10,5]

stats = pd.DataFrame()
for p in periods:
  model = calc_t_score(p,'Z-Score')

  period = str(p)+'D'

  constant_coef = model.params[0]
  sentiment_coef = model.params[1]

  constant_t = model.tvalues[0]
  sentiment_t = model.tvalues[1]


  r_squared = model.rsquared
  adj_r_squared = model.rsquared_adj

  data = {'Sentiment Coef': sentiment_coef,
          'Sentiment t-stat': sentiment_t,
          'Const. Coef' : constant_coef,
          'Const. t-stat' : constant_t,
          'r squared' : r_squared,
          'adj. r squared': adj_r_squared
  }

  df = pd.DataFrame(data, index=[period])

  stats = pd.concat([stats,df],axis=0)
  

In [ ]:
# Read all sentiment scores into list to calc std and mean
all = []
count = 0
for ticker in total_SP:
  temp = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Z-Scores/'+ticker+'.csv',index_col=0)
  print(ticker)
  count += len(temp)
  all.append(temp)

master = pd.concat(all, axis=0)
assert(len(master) == count)

<ipython-input-2-e9c3ba880cb5>:3: DtypeWarning: Columns (7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/master_minus_MD&A.csv',index_col=0)


In [ ]:
# Create and populate dictionary to sort tickers by GICS industry:

no_price_data = ['FRC', 'GEHC', 'SBNY']

industry_dict = {'Industrials':[],
             'Health Care':[],
             'Information Technology':[],
             'Communication Services':[],
             'Consumer Staples':[],
             'Consumer Discretionary':[],
             'Financials':[],
             'Materials':[],
             'Utilities':[],
             'Real Estate': [],
             'Energy':[]
             }

industry_csv = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/S&P-industries.csv',index_col=0)

for index, row in industry_csv.iterrows():
  sector = str(row['Sector'])
  ticker = str(row['Tickers'])
  if ticker in no_price_data:
    continue 

  industry_dict[sector].append(ticker)

# Veryify correct number of tickers - should be 500
keys = list(industry_dict.keys())
count = 0
for k in keys:
  count += len(industry_dict[k])

count

In [ ]:
def calc_z_scores(score_type):

  master = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/master.csv',index_col = 0)
  
  # Calculate sentiment score mean and std for winsorization
  scores = np.array(master[score_type])
  mean = np.mean(scores)
  std = np.std(scores)
  
  # Winsorize Scores to be maximally 3 stds from mean
  lower = mean - (3 * std)
  upper = mean + (3 * std)
  
  # winsorized_data = winsorize(scores, limits=(lower,upper))

  upperCount = 0
  lowerCount = 0

  # Winsorize outliers 
  for val in scores:
    if val > upper:
      val = upper
      upperCount +=1
    elif val < lower:
      val = lower
      lowerCount+=1
  
  print("UPPER COUNT: ", upperCount)
  print("LOWER COUNT: ", lowerCount)

  # Remove 0's to calculate mean and std for z scores
  no_zeros = scores[scores != 0]
  no_zero_mean = np.mean(no_zeros)
  no_zero_std = np.std(no_zeros)

  # Loop over winsorized data array to calculate a z score for each value
  z_scores = []
  for x in scores:
    z = (x - no_zero_mean) / no_zero_std
    z_scores.append(z)
  
  assert(len(z_scores) == len(master))

  if score_type == 'S-FLS':
    master['Z-Score-FLS'] = z_scores
  
  elif score_type == 'S-yiyang':
    master['Z-Score'] = z_scores

  master.to_csv('/content/drive/MyDrive/Capstone-DataSets/master-v1.1.csv')  

In [ ]:
def calc_t_score(period,score_type,industry):

  col = str(period) + ' Day Return'
  df = pd.DataFrame()

  if industry == 'All':
    df['Returns'] = master[col]
    df['ZScores'] = master[score_type]
  else:
    df = master[master['Ticker'].isin(list(industry_dict[industry]))]
    df['Returns'] = master[col]
    df['ZScores'] = master[score_type]

  print(len(df))

  df['Returns'] = pd.to_numeric(df.Returns, errors='coerce')
  df['ZScores'] = pd.to_numeric(df.ZScores, errors='coerce')
  df = df.dropna()

  print(len(df))


  y = np.array(df['Returns'])
  x = np.array(df['ZScores'])

  print(df)

  print(type(x))
  print(type(y))

  x = sm.add_constant(x)

  model = sm.OLS(y,x).fit()
  print(model.summary())
  # tvals = list(model.tvalues)

  # if len(tvals) == 1:
  #   return tvals[0]
  # else:
  #   return tvals[1]
  return model

In [ ]:
def calc_t_score_vol(period,score_type,industry):

  col = str(period) + 'D Vol'
  df = pd.DataFrame()

  if industry == 'All':
    df['Volatility'] = master[col]
    df['ZScores'] = master[score_type]
  else:
    df = master[master['Ticker'].isin(list(industry_dict[industry]))]
    df['Volatility'] = master[col]
    df['ZScores'] = master[score_type]

  print(len(df))

  df['Volatility'] = pd.to_numeric(df.Volatility, errors='coerce')
  df['ZScores'] = pd.to_numeric(df.ZScores, errors='coerce')
  df = df.dropna()

  print(len(df))


  y = np.array(df['Volatility'])
  x = np.array(df['ZScores'])

  print(df)

  print(type(x))
  print(type(y))

  x = sm.add_constant(x)

  model = sm.OLS(y,x).fit()
  print(model.summary())
  # tvals = list(model.tvalues)

  # if len(tvals) == 1:
  #   return tvals[0]
  # else:
  #   return tvals[1]
  return model

In [ ]:
# Method calculates a regression for each month beginning at the earliest month which information on 30 stocks is available
# Should start at 1994-04-01

# For each month at time t:
#   - Get most recent z score for every stock and put into one column (independent variable)
#   - Get corresponding 30 day return and put into another column (dependent variable)
#   - Perform a regression and record stats
# Plot sentiment coefficient overtime 

In [ ]:
class DateError(Exception):
    pass
def get_nearest_row(date,priceData):
  temp = priceData.loc[priceData['Date'] == date]
  if temp.empty:
    count = 0
    while temp.empty:
        date = pd.to_datetime(date)
        date = date + timedelta(days=1)
        date = date.strftime('%Y-%m-%d')
        temp = priceData.loc[priceData['Date'] == date]
        count += 1
        if count > 15:
          raise DateError('Matching Date not in dataset')
  return temp

In [ ]:
# def calc_rolling_regression():
from datetime import datetime
cur_date = '1994-04-01'
rolling_r = pd.DataFrame()

while cur_date <= '2023-03-01':

  print("CUR DATE: ", cur_date)
  

  # Iterate backwards in time over master DF until row date is less than or equal to cur_date
  # Get the most recent document for each stock by:
    # Add every row of master into a temp DF if and only if, a row with its ticker has not already been added
  # Because the master df is sorted, we know that the first row we hit for each company will be the most recent

  # df to hold independent and dependent variables in corresponding timeframe
  df = pd.DataFrame(columns=['Ticker','ZScore', 'Returns','Date'])

  # FILTER FOR INDUSTRY IF WANTED
  master_temp = master[master['Ticker'].isin(list(industry_dict['Utilities']))]


  # IF filtered for industry, MUST iterate over master_temp, not master
  for index, row in master_temp.iterrows():
    if row['Filed At'] <= cur_date:
      ticker = row['Ticker']
      tickers = list(df['Ticker'])
      # The first ticker added to df will be the most recent (with respect to cur_date), so if there is already an entry for a specific ticker, do nothing on continue to next row
      if ticker in tickers:
        continue
      else:
        # Get price at cur_date
        # mask = master_price_data['Ticker'] == ticker
        # price_data = master_price_data[mask]
        # price_data = price_data.reset_index(drop=True)
        price_data = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/PriceData/'+ticker+'.csv',index_col=0)
        price_data['Date'] = price_data['Date'].apply(lambda x: x[:10])
        print(cur_date, ticker)

        # If cur_date is before price history begins for specific stock, skip it.
        if cur_date < price_data.loc[0,'Date']:
          print(ticker," Skipped")
          continue

        # Get row from price data closest to cur_date
        price_row = get_nearest_row(cur_date,price_data)
        price_index = price_row.index[0]

        # Get price at cur_date and cur_date + 21 (trading) days to prepare percentage change calculation
        old_val = price_data.loc[price_index,'Adj Close']

        if (price_index + 21) > (len(price_data) - 1):
            monthly_return = 'NO DATA'
        else:
          new_val = price_data.loc[price_index+21,'Adj Close']

          # Calculate Monthly return between cur_date and cur_date + 21
          monthly_return = ((new_val - old_val)/old_val) * 100

        # Create df with single row to hold new calculations
        data = dict()
        print(row['Z-Score'])
        data = {'Ticker': row['Ticker'],
                'ZScore' : row['Z-Score'],
                'Returns' : monthly_return,
                'Date': row['Filed At']}
        temp = pd.DataFrame(data,index=[0])

        # Append new row to df containing total month information
        df = pd.concat([df,temp],axis=0)
        test = df
  
  # Prepare df for regression 
  # Remove rows with null values 
  df['Returns'] = pd.to_numeric(df.Returns, errors='coerce')
  df['ZScore'] = pd.to_numeric(df.ZScore, errors='coerce')
  df = df.dropna()

  # Construct independent and dependent variable vectors and perform regression
  y = np.array(df['Returns'])
  x = np.array(df['ZScore'])

  x = sm.add_constant(x)

  model = sm.OLS(y,x).fit()

  # Exctact specific fields from regression results and place in returned df
  constant_coef = model.params[0]
  sentiment_coef = model.params[1]

  constant_t = model.tvalues[0]
  sentiment_t = model.tvalues[1]


  r_squared = model.rsquared
  adj_r_squared = model.rsquared_adj
  
  data = dict()
  data = {'Date' : cur_date,
          'Sentiment Coef': sentiment_coef,
          'Sentiment t-stat': sentiment_t,
          'Const. Coef' : constant_coef,
          'Const. t-stat' : constant_t,
          'r squared' : r_squared,
          'adj. r squared': adj_r_squared
  }

  df = pd.DataFrame(data,index=[0])
  rolling_r = pd.concat([rolling_r,df],axis=0)

  date = datetime.strptime(cur_date, '%Y-%m-%d')
  cur_date = date + relativedelta(months=1)
  cur_date = cur_date.strftime('%Y-%m-%d')


In [ ]:
def getPercentReturn(old_val,new_val):

    percent_change = ((new_val - old_val)/old_val) * 100
    return percent_change


def calc_volatility(df,date,period):
  # Get row (from price data) with closest date to filing date
  # If no matching row in price data, return nan
  try:
    row = get_nearest_row(date,df)
  except DateError:
    return np.nan
  price_index = row.index[0]
  
  # Array returns will hold daily returns over given period
  returns = np.array([])
  count = 0
  while count < period:
    old_val = df.loc[price_index,'Adj Close']
    if price_index + 1 > len(df)-1:
      break

    new_val = df.loc[price_index+1,'Adj Close']
    returns = np.append(returns,getPercentReturn(old_val,new_val))

    count+=1
    price_index +=1
  
  volatility = np.std(returns)

  return volatility

In [ ]:
def calc_reg_plot(period,score_type,industry):

  col = str(period) + ' Day Return'
  df = pd.DataFrame()

  if industry == 'All':
    df['Returns'] = master[col]
    df['ZScores'] = master[score_type]
  else:
    df = master[master['Ticker'].isin(list(industry_dict[industry]))]
    df['Returns'] = master[col]
    df['ZScores'] = master[score_type]

  print(len(df))

  df['Returns'] = pd.to_numeric(df.Returns, errors='coerce')
  df['ZScores'] = pd.to_numeric(df.ZScores, errors='coerce')
  df = df.dropna()

  print(len(df))


  y = np.array(df['Returns'])
  x = np.array(df['ZScores'])

  # sns.regplot(x=x, y=y,scatter_kws={'s': 0.05},line_kws={"color":"r", "linewidth":0.5})
  # plt.xlabel("Adj. Sentiment Score")
  # plt.ylabel("Return")
  # plt.axhline(y=0, linestyle='-.', color='yellow', "linewidth":0.5})

  sns.regplot(x=x, y=y, scatter_kws={'s': 0.05}, line_kws={"color": "r", "linewidth": 1.5})

  # Add x and y axis labels
  plt.xlabel("Adj. Sentiment Score")
  plt.ylabel("Return")
  plt.title('365D Return Regression')
  plt.ylim(top=300)

  # Add a horizontal dashed line at y=0 with yellow color and 0.5 line width
  plt.axhline(y=0, linestyle='--', color='r', linewidth=1)
  # remove the box around the graph
  plt.gca().spines['top'].set_visible(False)
  plt.gca().spines['right'].set_visible(False)
  plt.gca().spines['bottom'].set_visible(False)
  plt.gca().spines['left'].set_visible(False)
  # Show the plot
  plt.show()

In [ ]:
time_periods = np.array(['365D', '180D', '90D', '60D', '30D', '10D','5D'])
returns = np.array([-16.7, -12, -10, -10.5, -9.9, -6,-1.98])
volatility = np.array([-24.5, -30.3, -32.6, -33, -32.3, -30.7,0])

# Convert time_periods to categorical type
cat_time_periods = pd.Categorical(time_periods, categories=time_periods)

# Create a figure and axis object
fig, ax = plt.subplots()
ax.grid(color='lightgray', alpha=0.7)

# Set the width of the bars
bar_width = 0.25

# Create the bar plot for returns
ax.bar(cat_time_periods.codes, returns, width=bar_width, label='Return', zorder=2)

# Shift the position of the bars for volatility
ax.bar(cat_time_periods.codes + bar_width, volatility, width=bar_width, label='Volatility', zorder=2)

# Set the labels and title for the plot
ax.set_xlabel('Period')
ax.set_ylabel('T-statistic value')


# Set the xticks to the categorical labels
ax.set_xticks(cat_time_periods.codes)
ax.set_xticklabels(cat_time_periods.categories)

# Add a dashed red line to the plot
ax.axhline(y=-2, color='red', linestyle='-.', label='Significance Threshold |t| > 2')

# Add a legend to the plot
ax.legend(loc='center right', bbox_to_anchor=(1.3, 1.3))

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the data to be plotted
industries = ['Energy', 'Materials', 'Industrials', 'Consumer Discretionary', 
              'Consumer Staples', 'Health Care', 'Financials', 'Information Technology', 
              'Communication Services', 'Utilities', 'Real Estate','Total']
mean_values = [-2.67, -0.68, -8, -2.2, -2.37, -3.55, -0.18, -1.99, -1.82, -0.66, -6.29,-9.57]  # your own list of mean values
median_values = [-2.48, -3.28, -7.1, -2.64, -2.54, -3.77, -0.12, -2.24, -1.39, -1.01, -6.31,-9.97]  # your own list of median values

# Set up the figure and axis with a larger size
fig, ax = plt.subplots(figsize=(12, 6))

# Define the width of each bar
bar_width = 0.35

# Plot the bars for each industry
x_pos = np.arange(len(industries))
ax.bar(x_pos - bar_width/2, mean_values, width=bar_width, label='Mean')
ax.bar(x_pos + bar_width/2, median_values, width=bar_width, label='Median')

# Set the axis labels and title
ax.set_xlabel('Industry')
ax.set_ylabel('Value')
ax.set_title('Mean and Median Values by Industry')

# Set the x-axis tick marks and labels
ax.set_xticks(x_pos)
ax.set_xticklabels(industries, rotation=45, ha='right')

# Add a legend
ax.legend()

# Show the plot
plt.show()

In [ ]:
# Define the data to be plotted
industries = ['Energy', 'Materials', 'Industrials', 'Consumer Discretionary', 
              'Consumer Staples', 'Health Care', 'Financials', 'Information Technology', 
              'Communication Services', 'Utilities', 'Real Estate','Total']


returns = [-2.67, -0.68, -8, -2.2, -2.37, -3.55, -0.18, -1.99, -1.82, -0.66, -6.29,-9.57]  # your own list of mean values
volatility = [-2.23, -5.8, -9.43, -16.72, -4.12, -12.73, -10.27, -16.59, -4.46, -3.9, -9.76,-30.54]  # your own list of median values

# Sort the data by mean values in descending order
sorted_data = sorted(zip(industries, returns, volatility), key=lambda x: x[2], reverse=False)
industries, returns, volatility = zip(*sorted_data)

# Set up the figure and axis with a larger size
fig, ax = plt.subplots(figsize=(12, 6))

# Define the width of each bar
bar_width = 0.35

# Plot the bars for each industry
x_pos = np.arange(len(industries))
ax.bar(x_pos - bar_width/2, returns, width=bar_width, label='Return',zorder=2)
ax.bar(x_pos + bar_width/2, volatility, width=bar_width, label='Volatility',zorder=2)

# Set the axis labels and title
ax.set_ylabel('T-statistic value')


# Set the x-axis tick marks and labels
ax.set_xticks(x_pos)
ax.set_xticklabels(industries, rotation=45, ha='right')

# Add a legend
ax.legend(framealpha=1)
ax.grid(True, linestyle='--', color='gray', alpha=0.5, zorder=1)

# Show the plot
plt.show()

In [ ]:
time_periods = np.array(['365D', '180D', '90D', '60D', '30D', '10D','5D'])
returns = np.array([-16.7, -12, -10, -10.5, -9.9, -6,-1.98])
volatility = np.array([-24.5, -30.3, -32.6, -33, -32.3, -30.7,0])
ma = np.array([-16.03, -10.66, -7.83, -8.09, -8.1, -5, -1.31])  # new array for "MA" values

# Convert time_periods to categorical type
cat_time_periods = pd.Categorical(time_periods, categories=time_periods)

# Create a figure and axis object
fig, ax = plt.subplots()
ax.grid(color='lightgray', alpha=0.7)

# Set the width of the bars
bar_width = 0.25

# Create the bar plot for returns
ax.bar(cat_time_periods.codes, returns, width=bar_width, label='Return', zorder=2)

# Shift the position of the bars for volatility
ax.bar(cat_time_periods.codes + bar_width, volatility, width=bar_width, label='Volatility', zorder=2)

# Add a new bar plot for MA values
ax.bar(cat_time_periods.codes + 2*bar_width, ma, width=bar_width, label='Return (MA Dif)', zorder=2)

# Set the labels and title for the plot
ax.set_xlabel('Period')
ax.set_ylabel('T-statistic value')


# Set the xticks to the categorical labels
ax.set_xticks(cat_time_periods.codes + bar_width)
ax.set_xticklabels(cat_time_periods.categories)

# Add a dashed red line to the plot
ax.axhline(y=-2, color='red', linestyle='-.', label='Significance Threshold |t| > 2')

# Add a legend to the plot
ax.legend(loc='center right', bbox_to_anchor=(1.3, 1.3))

# Display the plot
plt.show()

# Sentiment Analysis


In [ ]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [ ]:
# Load and configure finBERT model to run on instance GPU. This portion of the code was executed using a Nvidia V100 via
# a custom Google GCE VM.
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3).to("cuda")
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',model_max_length=512)
classifier = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer,device=0,batch_size=32)
print_gpu_utilization()

In [ ]:
def get_sentiment(ticker):
  filing = pd.read_csv('/content/Filings/'+ticker+'.csv',index_col=0,encoding='utf-8')
  filing = filing.dropna()
  filing = filing.reset_index(drop=True)
  filing['MD&A'] = filing['MD&A'].replace(r'\n',' ',regex=True)
  filing['S-yiyang'] = None


  # finbert Sentence Level Granularity
  for index, row in filing.iterrows():  

    # Tokenize mdna section into individual sentences
    text = filing['MD&A'][index]
    sentences = nltk.sent_tokenize(text)

    # Initalize counters
    neg_count = 0
    neutral_count = 0
    pos_count = 0

    for s in sentences:
      # Get sentiment of sentence
      sentiment = classifier(s,padding=True, truncation=True)

      # Count occurrences of pos/neutral/neg
      if sentiment[0]['label'] == 'Positive':
        pos_count = pos_count + 1

      if sentiment[0]['label'] == 'Neutral':
        neutral_count = neutral_count + 1

      if sentiment[0]['label'] == 'Negative':
        neg_count = neg_count + 1

    # Calculate final sentiment score
    total = neg_count + neutral_count + pos_count
    percent_neg = neg_count / total
    percent_pos = pos_count / total
    sentiment_score = (percent_pos - percent_neg)



    # Push score to DF 
    filing.loc[index,'S-yiyang'] = sentiment_score
    print(index," ", ticker, " ",sentiment_score)
  
  # filing['S-yiyang'] = scores
  filing.to_csv('/content/Filings_and_Sentiment/'+ticker+'-S.csv')

In [ ]:
# Verify all files have been downloaded
todo = []
for ticker in total_SP:
  try:
    test = pd.read_csv('/content/drive/MyDrive/Capstone-DataSets/Two_Sentiment/'+ticker+'-2C.csv')
  except FileNotFoundError:
    todo.append(ticker)
    print(ticker)